In [2]:
import jsonlines
import torchaudio
from pathlib import Path
import torch, random
import librosa, os
import IPython.display as ipd
from dotenv import load_dotenv
from spellchecker import SpellChecker
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
from collections import defaultdict
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
import os

import torch
import torchvision
from torchvision import ops
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [3]:
import jsonlines
import pandas as pd

data = {'image': [], 'caption': [], 'bbox': []}
load_dotenv()

TEAM_NAME = os.getenv("TEAM_NAME", "7up")
TEAM_TRACK = os.getenv("TEAM_TRACK", "advanced")


input_dir = Path(f"/home/jupyter/{TEAM_TRACK}")
with jsonlines.open(input_dir / "vlm.jsonl") as reader:
    for obj in reader:
        image = obj['image']
        annotations = obj['annotations']
        for annotation in annotations:
            caption = annotation['caption']
            bbox = annotation['bbox']
            data['image'].append(image)
            data['caption'].append(caption)
            data['bbox'].append(bbox)

df = pd.DataFrame(data)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
df.head()

,image,caption,bbox
0,image_0.jpg,grey missile,"[912, 164, 48, 152]"
1,image_0.jpg,"red, white, and blue light aircraft","[1032, 80, 24, 28]"
2,image_0.jpg,green and black missile,"[704, 508, 76, 64]"
3,image_0.jpg,white and red helicopter,"[524, 116, 112, 48]"
4,image_1.jpg,grey camouflage fighter jet,"[1112, 172, 64, 36]"


In [4]:
unique_values = df['caption'].unique()

In [5]:
from torchvision import transforms

data_transforms = transforms.Compose([
    # transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [6]:
class ObjectDetectionDataset(Dataset):
    '''
    A Pytorch Dataset class to load the images and their corresponding annotations.
    
    Returns
    ------------
    images: torch.Tensor of size (B, C, H, W)
    gt bboxes: torch.Tensor of size (B, max_objects, 4)
    gt classes: torch.Tensor of size (B, max_objects)
    '''
    def __init__(self, df, img_dir, img_size, name2idx, transforms=None):
        self.df = df
        self.image = df["image"]
        self.caption = df["caption"]
        self.bbox = df["bbox"]
        self.img_dir = img_dir
        self.img_size = img_size
        self.name2idx = name2idx
        self.transforms = transforms
        self.img_data_all, self.gt_bboxes_all, self.gt_classes_all = self.get_data()
        
    def __len__(self):
        return self.img_data_all.size(dim=0)
    
    def __getitem__(self, idx):
        return self.img_data_all[idx], self.gt_bboxes_all[idx], self.gt_classes_all[idx]
        
    def get_data(self):
        img_data_all = []
        gt_boxes_all = []
        gt_classes_all = []

        for i, img_path in enumerate(self.image):
            full_img_path = os.path.join(self.img_dir, img_path)

            img = Image.open(full_img_path).convert("RGB")
            if self.transforms:
                img = self.transforms(img)
            else:
                img = transforms.ToTensor()(img)
            # print(img)
            img_data_all.append(img)

            gt_bboxes = self.bbox[i]
            gt_classes = self.caption[i]
            # print(gt_bboxes)
            # print(gt_classes)
            gt_class_indices = torch.tensor(self.name2idx[gt_classes], dtype=torch.long)
            
            # print(gt_class_indices)
            gt_boxes_all.append(torch.tensor(gt_bboxes, dtype=torch.float32))
            gt_classes_all.append(gt_class_indices)

        gt_bboxes_pad = pad_sequence(gt_boxes_all, batch_first=True, padding_value=-1)
        gt_classes_pad = pad_sequence(gt_classes_all, batch_first=True, padding_value=-1)

        img_data_stacked = torch.stack(img_data_all, dim=0)
        
        return img_data_stacked, gt_bboxes_pad, gt_classes_pad

In [7]:
img_width = 1520
img_height = 870
df_input = df
image_dir = "/home/jupyter/advanced/images/"
name2idx = {value: idx for idx, value in enumerate(unique_values)}
name2idx['pad'] = -1
idx2name = {v:k for k, v in name2idx.items()}
idx2name

{0: 'grey missile',
 1: 'red, white, and blue light aircraft',
 2: 'green and black missile',
 3: 'white and red helicopter',
 4: 'grey camouflage fighter jet',
 5: 'grey and white fighter plane',
 6: 'white and black drone',
 7: 'white and black fighter jet',
 8: 'white missile',
 9: 'black and white commercial aircraft',
 10: 'grey drone',
 11: 'yellow, red, and grey helicopter',
 12: 'yellow commercial aircraft',
 13: 'black cargo aircraft',
 14: 'yellow helicopter',
 15: 'white and black light aircraft',
 16: 'grey and black fighter plane',
 17: 'red fighter plane',
 18: 'blue helicopter',
 19: 'white, red, and green fighter plane',
 20: 'black camouflage fighter jet',
 21: 'green light aircraft',
 22: 'blue and yellow fighter jet',
 23: 'white fighter jet',
 24: 'red fighter jet',
 25: 'blue and white light aircraft',
 26: 'white and black helicopter',
 27: 'white and blue fighter plane',
 28: 'grey commercial aircraft',
 29: 'blue and grey fighter jet',
 30: 'green and brown camo

In [ ]:
od_dataset = ObjectDetectionDataset(df_input, image_dir, (img_height, img_width), name2idx, data_transforms)


In [8]:
od_dataloader = DataLoader(od_dataset, batch_size=2)


NameError: name 'od_dataset' is not defined

In [ ]:
for img_batch, gt_bboxes_batch, gt_classes_batch in od_dataloader:
    img_data_all = img_batch
    gt_bboxes_all = gt_bboxes_batch
    gt_classes_all = gt_classes_batch
    break
    
img_data_all = img_data_all[:2]
gt_bboxes_all = gt_bboxes_all[:2]
gt_classes_all = gt_classes_all[:2]

In [ ]:
# get class names
gt_class_1 = gt_classes_all[0].long()
gt_class_1 = [idx2name[idx.item()] for idx in gt_class_1]

gt_class_2 = gt_classes_all[1].long()
gt_class_2 = [idx2name[idx.item()] for idx in gt_class_2]

In [ ]:
nrows, ncols = (1, 2)
fig, axes = plt.subplots(nrows, ncols, figsize=(16, 8))

fig, axes = display_img(img_data_all, fig, axes)
fig, _ = display_bbox(gt_bboxes_all[0], fig, axes[0], classes=gt_class_1)
fig, _ = display_bbox(gt_bboxes_all[1], fig, axes[1], classes=gt_class_2)